<center><b><font size=6>Lab-6 A classifier from scratch<b><center>

### Objective: Implement, use and evaluate a classifier (without using specific libraries such as sklearn)
1. **Logistic regression** is a binary classification method that maps a linear combination of parameters and variables into two possible classes. Here, you will implement the logistic regression from scratch to better understand how an ML algorithm works. Useful link: <a href="https://en.wikipedia.org/wiki/Logistic_regression">Wiki</a>.
2. **Performance evaluation metrics** are needed to evaluate the outcome of prediction with respect to true labels. Here, you will implement confusion matrix, accuracy, precision, recall and F-measure. Useful link: <a href="https://en.wikipedia.org/wiki/Confusion_matrix">Wiki</a>.

In [3]:
# import needed python libraries

%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random

### 1. Dataset - TCP logs
The dataset contains traffic information generated by an open-source passive network monitoring tool, namely **tstat**. It automates the collection of packet statistics of traffic aggregates, using real-time monitoring features. Being a passive tool, the typical usage scenario is live monitoring of Internet links, in which all transmitted packets are observed. In case of TCP, Tstat identifies a new flow start when it observes a TCP three-way handshake. Similarly, it identifies a TCP flow end either when it sees the TCP connection teardown, or when it doesn’t observe packets for some time (idle time). A flow is defined by a unique link between the sender and receiver, e.g., a tuble of <em>(IP_Protocol_Type, IP_Source_Address, Source_Port, IP_Destination_Address, Destination_Port)</em>. For a specific flow, tstat calculates a number of statistics of all the packets transmitted over this flow, and then generate a log for such flow with multiple attributes (statistics). A log file is arranged as a simple table where each column is associated to specific information and each row reports the flow during a connection. The log information is a summary of the flow properties. For instance, in the TCP log we can find columns like the starting time of a TCP connection, its duration, the number of sent and received packets, the observed Round Trip Time.
![](tstat.png)

In this lab, since the focus is on the development of logistic regression from scratch, we only consider a portion of the dataset for simplicity. The data can be found in `log_tcp_part.csv`, in which there are multiple columns, the last one is the class label, indicating the flow is from either **google** or **youtube**, and the rest are features. Your job is a binary classification task to classify the domain of each flow (row) **from scratch**, including:
- Build a logistic regression model,
- Evaluate the performance.

1. Load the dataset.
2. Get the list of features (columns 1 to 10).
3. Add a new column and assign numerical class labels of -1 and 1 to google and youtube.
4. Answering the following questions:
    - How many features do we have?
    - How many samples do we have in total?
    - How many samples do we have for each class? Are they similar?

In [4]:
df = pd.read_csv('log_tcp_part.csv')
df['class_label'] = None
df['class_label'].loc[df['class'] == 'google'] = -1
df['class_label'].loc[df['class'] == 'youtube'] = 1

display(df)
samples = df.shape[0]
print(f"Samples: {samples}")
print(f"Features: {df.shape[1]-2}")
print(f"Youtube: {df['class_label'].value_counts()[1]}")
print(f"Google: {df['class_label'].value_counts()[-1]}")

X = df[df.columns[:10]]
y = df[df.columns[11]]

print(X)


/tmp/ipykernel_13996/2480156299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class_label'].loc[df['class'] == 'google'] = -1
/tmp/ipykernel_13996/2480156299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class_label'].loc[df['class'] == 'youtube'] = 1


,c_msgsize_count,c_pktsize6,c_msgsize4,s_msgsize4,s_pktsize2,s_rtt_cnt,s_rtt_std,s_msgsize5,c_msgsize6,c_sit3,class,class_label
0,1,0,0,0,1418,0,0.000000,0,0,0.000,google,-1
1,1,0,0,0,0,3,0.466732,0,0,0.000,google,-1
2,1,0,0,0,0,3,0.413304,0,0,0.000,google,-1
3,1,0,0,0,1418,1,0.000000,0,0,0.000,google,-1
4,1,0,0,0,1418,0,0.000000,0,0,0.000,google,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,4,0,37,0,1418,3,22.224528,0,0,3.334,youtube,1
19996,6,45,45,57,1418,2,0.000000,45,45,1.252,youtube,1
19997,4,0,1205,0,531,4,15.323660,0,0,4975.694,youtube,1
19998,4,0,690,0,767,4,17.997651,0,0,1719.125,youtube,1


Samples: 20000
Features: 10
Youtube: 10000
Google: 10000
       c_msgsize_count  c_pktsize6  c_msgsize4  s_msgsize4  s_pktsize2  \
0                    1           0           0           0        1418   
1                    1           0           0           0           0   
2                    1           0           0           0           0   
3                    1           0           0           0        1418   
4                    1           0           0           0        1418   
...                ...         ...         ...         ...         ...   
19995                4           0          37           0        1418   
19996                6          45          45          57        1418   
19997                4           0        1205           0         531   
19998                4           0         690           0         767   
19999                1           0           0           0           0   

       s_rtt_cnt  s_rtt_std  s_msgsize5  c_msgsize6   

### 2. Implement your logistic regression learning algorithm
Here you will need to construct a class in which you need to define two functions besides the class initialization:
- `fit`. In this method you will perform ERM. Learn the parameters of the model (i.e., the hypothesis h) from training with gradient descent
- `predict`. In this method given one  sample x (or more) you will perform the inference $sign(h(x))$ to obtain class labels.

Hints:

- The linear function used in the logistic regression is the following: $h(x)=w^T x +b $, where b is a scalar bias.
- Logistic loss: $L((x,y),h)=\log(1+e^{-y h(x)})$
- ERM: $\min_{w,b} f(w,b)=\frac{1}{m}\sum_{i=1}^{m} \log(1+e^{-y^{(i)} h(x^{(i)})})$
- Gradient for weight: $\nabla_w f(w,b) = \frac{1}{m} \sum_i \frac{-y^{(i)}x^{(i)}}{(1+e^{-y^{(i)}h(x^{(i)})})}$
- Gradient for bias: $\nabla_b f(w,b)= \frac{1}{m} \sum_i \frac{-y^{(i)}}{(1+e^{-y^{(i)}h(x^{(i)})})}$
- Update the parameters: $w \leftarrow w - \alpha \nabla w$, $b \leftarrow b - \alpha  \nabla b$

Notice that the sigmoid function $f(z) = \frac{1}{1 + e^{-z}}$ appears multiple times. You can write also a method for the sigmoid function to help you in the computation. By considering f(z), the gradients rewrite as:

- Gradient for weight: $\nabla_w f(w,b) = \frac{1}{m} \sum_i ({f(h(x^{(i)})) - y^{(i)}})x^{(i)}$
- Gradient for bias: $\nabla_b f(w,b) = \frac{1}{m} \sum_i ({f(h(x^{(i)})) - y^{(i)}})$

In [5]:
class LogisticRegression:
    def __init__(self, learning_rate, num_iterations):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
    
    def inizialize_weights(self, dim):
        w = np.zeros_like(dim)
        b = 0
        return w,b
    
    def logloss(self, y_true, y_pred):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        log_loss = -1 * np.mean( y_true*np.log10(y_pred) + (1-y_true)*np.log10(1-y_pred)) 
        return log_loss

    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def gradient_dw(self, x, y, w, b, alpha, N):
        dw = x * (y-self.sigmoid(np.dot(w.T,x)+b)) - ((alpha*w*w)/N)
        return dw
    

### 3. Use the model
- Initialize your model with predefined learning rate of `0.1` and iterations of `100`.
- Fit your model with features and targets.
- Get the prediction with features.

In [ ]:
# your answers here

### 4. Model evaluation
With predicted class labels and ground truths, we now evaluate the model performance through confusion matrix and numerical metrics. Specifically, you need to derive the following:
- Confusion matrix - Note that, you should indicate the corresponding quantity of each element in the table. Here positive is class 0 and negative is class 1:
\begin{array}{|c|c|c|}
\hline
 & \textbf{Predicted Positive} & \textbf{Predicted Negative} \\
\hline
\textbf{Actual Positive} & \text{True Positive (TP)} & \text{False Negative (FN)} \\
\hline
\textbf{Actual Negative} & \text{False Positive (FP)} & \text{True Negative (TN)} \\
\hline
\end{array}
- Precision of each class and the average value:
$\frac{\text{True Positive (TP)}}{\text{True Positive (TP) + False Positive (FP)}}$
- Recall of each class and the average value:
$\frac{\text{True Positive (TP)}}{\text{True Positive (TP) + False Negative (FN)}}$
- F1-score of each class and the average value:
$F_1 = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$
- Accuracy:
$\frac{\text{True Positive (TP) + True Negative (TN)}}{\text{True Positive (TP) + True Negative (TN) + False Positive (FP) + False Negative (FN)}}$
- Answering the following questions:
    - Do you have same performance between classes? If not, which one performs better?
    - Change the parameters of learning rate or number of iterations. Do you have same performance? Better or Worse? Why?

In [ ]:
# your answers here